In [1]:
import torch
from utils import get_predictions
import os
from utils import get_data,preprocess_image,plot_roc_curve
import pandas as pd
from models import model_dict,select_model
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np

In [2]:

dataset_total = pd.read_csv("C:\\Users\\PC\\Desktop\\diabetic retinopathy\\data\\labels\\label.csv")
validation_directiory = "C:\\Users\\PC\\Desktop\\diabetic retinopathy\\data\\validation_images"
train_test_image_directory = "C:\\Users\\PC\\Desktop\\diabetic retinopathy\\data\\test_train_images"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

files_and_dirs = os.listdir("C:\\Users\\PC\\Desktop\\diabetic retinopathy\\saved models") 



In [3]:
all_predictions = []
weights = []

for items in files_and_dirs:
    model_name, file_extension = os.path.splitext(items)

    _, _, valid_data = get_data(dataset_total, train_test_image_directory, validation_directiory,
                                train_test_sample_size=10, batch_size=16,
                                image_filter=preprocess_image, model=model_name, validation=True)

    state_dict = torch.load(f"C:\\Users\\PC\\Desktop\\diabetic retinopathy\\saved models\\{items}")

    model_instance = select_model(model_name)

    model_instance = model_instance.to(device)

    model_instance.load_state_dict(state_dict)

    weight = model_dict[model_name]["weight"]
    weights.append(weight)

    predictions = get_predictions(model_instance, valid_data, device)
    
    all_predictions.append(predictions)

2023-07-28 23:19:49,779 [INFO] - Starting data preparation
2023-07-28 23:19:49,786 [INFO] - Balancing the train-test dataset
2023-07-28 23:19:49,788 [INFO] - Setting up data transforms
2023-07-28 23:19:49,789 [INFO] - Preparing Validation DataLoader
2023-07-28 23:19:49,790 [INFO] - Data preparation completed
2023-07-28 23:20:22,900 [INFO] - Starting data preparation
2023-07-28 23:20:22,907 [INFO] - Balancing the train-test dataset
2023-07-28 23:20:22,909 [INFO] - Setting up data transforms
2023-07-28 23:20:22,910 [INFO] - Preparing Validation DataLoader
2023-07-28 23:20:22,912 [INFO] - Data preparation completed
2023-07-28 23:20:56,402 [INFO] - Starting data preparation
2023-07-28 23:20:56,407 [INFO] - Balancing the train-test dataset
2023-07-28 23:20:56,410 [INFO] - Setting up data transforms
2023-07-28 23:20:56,411 [INFO] - Preparing Validation DataLoader
2023-07-28 23:20:56,412 [INFO] - Data preparation completed
2023-07-28 23:21:34,298 [INFO] - Starting data preparation
2023-07-28 

thisone


2023-07-28 23:22:03,740 [INFO] - Starting data preparation
2023-07-28 23:22:03,747 [INFO] - Balancing the train-test dataset
2023-07-28 23:22:03,748 [INFO] - Setting up data transforms
2023-07-28 23:22:03,749 [INFO] - Preparing Validation DataLoader
2023-07-28 23:22:03,751 [INFO] - Data preparation completed


thisone


2023-07-28 23:22:32,576 [INFO] - Starting data preparation
2023-07-28 23:22:32,583 [INFO] - Balancing the train-test dataset
2023-07-28 23:22:32,586 [INFO] - Setting up data transforms
2023-07-28 23:22:32,586 [INFO] - Preparing Validation DataLoader
2023-07-28 23:22:32,588 [INFO] - Data preparation completed


...

thisone


2023-07-28 23:23:18,895 [INFO] - Starting data preparation
2023-07-28 23:23:18,901 [INFO] - Balancing the train-test dataset
2023-07-28 23:23:18,904 [INFO] - Setting up data transforms
2023-07-28 23:23:18,905 [INFO] - Preparing Validation DataLoader
2023-07-28 23:23:18,906 [INFO] - Data preparation completed


thisone


2023-07-28 23:23:43,244 [INFO] - Starting data preparation
2023-07-28 23:23:43,250 [INFO] - Balancing the train-test dataset
2023-07-28 23:23:43,254 [INFO] - Setting up data transforms
2023-07-28 23:23:43,254 [INFO] - Preparing Validation DataLoader
2023-07-28 23:23:43,256 [INFO] - Data preparation completed


thisone


In [4]:
all_predictions = list(zip(*all_predictions))

ensemble_preds = []
for preds in all_predictions:
    weighted_votes = Counter()
    for pred, weight in zip(preds, weights):
        weighted_votes[pred] += weight
    ensemble_preds.append(weighted_votes.most_common(1)[0][0])

predicted_labels_list = [int(label.item()) for label in ensemble_preds]

true_labels = []
for _, labels in valid_data:
    true_labels.extend(labels.numpy())

In [8]:
true_labels

[0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 2,
 1,
 1,
 0,
 3,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 4,
 2,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 3,
 0,
 0,
 2,
 1,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 2,
 3,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 0,


In [5]:
def evaluate_classification_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)

    return {
        'accuracy': accuracy,
        'f1_score': f1,
        'precision': precision,
        'recall': recall,
        'confusion_matrix': conf_matrix
    }



In [6]:
predicted_labels_list

[0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 4,
 2,
 2,
 2,
 2,
 0,
 1,
 3,
 0,
 2,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 2,
 4,
 3,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 1,
 0,
 2,
 4,
 0,
 0,
 2,
 0,
 1,
 2,
 1,
 0,
 3,
 3,
 2,
 0,
 0,
 0,
 1,
 1,
 2,
 0,
 2,
 0,
 1,
 0,
 2,
 4,
 1,
 0,
 0,
 2,
 3,
 2,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 0,
 4,
 1,
 0,
 2,
 0,
 3,
 2,
 0,
 0,
 3,
 1,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 3,
 2,
 1,
 0,
 1,
 0,
 1,
 2,
 4,
 2,
 4,
 4,
 4,
 0,
 1,
 2,
 0,
 2,
 2,
 0,
 0,
 0,
 4,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 3,
 2,
 4,
 2,
 2,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 3,
 2,
 2,
 3,
 2,
 1,
 1,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 3,
 2,
 0,
 0,
 2,
 2,
 1,
 2,
 1,
 0,
 3,
 2,
 2,
 0,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 3,
 1,
 2,
 0,
 3,
 0,
 0,
 1,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,


In [7]:
evaluate_classification_metrics(true_labels,predicted_labels_list)

{'accuracy': 0.288,
 'f1_score': 0.3384334981156622,
 'precision': 0.4848086716314974,
 'recall': 0.288,
 'confusion_matrix': array([[109,  58, 137,  26,  12],
        [ 33,  13,  30,   6,   0],
        [ 15,  10,  19,   3,   3],
        [  6,   2,   4,   1,   0],
        [  4,   3,   4,   0,   2]], dtype=int64)}